In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

Define the learning model VGG16

In [3]:
def VGG16():
    model = Sequential()
    # Block 1
    model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same', input_shape = (48, 48, 1)))
    model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    # Block 2
    model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    # Block 3
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(256, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    # Block 4
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    #Block 5
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    #Flatten
    model.add(Flatten())
    model.add(Dropout(0.5))
    
    #Fully connected
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation = 'relu'))
    model.add(Dropout(0.5))
    
    #Output
    model.add(Dense(7, activation = 'softmax'))
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(0.0005), metrics=['accuracy'])
    return model

model, model_mask = VGG16(), VGG16()

Load the dataset we need and regions we want to test

In [4]:
#Loading the region for the left eye
left_eye = np.load('../input/valid-fer/left_eye.npy')
left_eye = left_eye.astype(int)

#Loading the region for the right eye
right_eye = np.load('../input/valid-fer/right_eye.npy')
right_eye = right_eye.astype(int)

#Loading the region for the nose
nose = np.load('../input/valid-fer/nose.npy')
nose = nose.astype(int)

#Loading the region for the mouth
mouth = np.load('../input/valid-fer/mouth.npy')
mouth = mouth.astype(int)

#Loading the dataset
valid_array = np.load('../input/valid-array/valid_array (4).npy')
valid_label = np.load('../input/valid-fer/valid_label.npy')

X = valid_array.astype('float32')/255

#Convert class vetcors to class matrices
y = to_categorical(valid_label)

Testing

In [5]:
#Fitting parameters
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 15, restore_best_weights = True)

fit_params = {'callbacks': [es],
              'epochs': 5,
              'batch_size': 32,
              'validation_split': .2,
              'verbose': 0}

split_params = {'split': 'one-split',
                'num_perm': 100,                
                'perturb_scale': 5,
                'min_inf': 0,
                'min_est': 0,
                'cv_num': 2,
                'verbose': 1}

In [21]:
#Inference based on dnn_inference
from BBoxTest import ada_split_test

#Test for eyes
print('Testing for eyes')
cue_eye = ada_split_test(model = model, 
                         model_mask = model_mask, 
                         change = 'mask', 
                         eva_metric = 'zero-one')

#Generate Z for eyes region
Z_eye = cue_eye.mask_cov(X, [left_eye, right_eye])
#Check whether the region is correct
fig, axs = plt.subplots(1, 4, figsize = (25, 12))
fig.subplots_adjust(hspace = 0.2, wspace = 0.2)
axs = axs.ravel()

picture = [Z_eye[10], Z_eye[20], Z_eye[30], Z_eye[40]]
for i in range(4):
    axs[i].imshow(picture[i][:,:,0], cmap = 'gray')
    axs[i].set_xticklabels([])
    axs[i].set_yticklabels([])

#Generate the splitting ratio and perturb level for eyes
ratio_eye, perturb_eye = cue_eye.hp_tuning(X = X, 
                                           Z = Z_eye, 
                                           y = y, 
                                           inf_feats = [left_eye, right_eye], 
                                           fit_params = fit_params, 
                                           split_params = split_params)

#Do the adaptive testing to get the p-value for the eye region
p_value_tmp_eye = cue_eye.ada_testing(X = X, 
                                      Z = Z_eye, 
                                      y = y, 
                                      ratio = ratio_eye, 
                                      perturb = perturb_eye, 
                                      fit_params = fit_params, 
                                      split_params = split_params)

In [ ]:
#Test for nose
print('Testing for nose')
cue_nose = split_test(model = model, 
                      model_mask = model_mask, 
                      change = 'mask', 
                      eva_metric = 'zero-one')

#Generate Z for nose region
Z_nose = cue_nose.mask_cov(X, nose)
#Check whether the region is correct
fig, axs = plt.subplots(1, 4, figsize = (25, 12))
fig.subplots_adjust(hspace = 0.2, wspace = 0.2)
axs = axs.ravel()

picture = [Z_nose[10], Z_nose[20], Z_nose[30], Z_nose[40]]
for i in range(4):
    axs[i].imshow(picture[i][:,:,0], cmap = 'gray')
    axs[i].set_xticklabels([])
    axs[i].set_yticklabels([])

#Generate the splitting ratio and perturb level for nose
ratio_nose, perturb_nose = cue_nose.hp_tuning(X = X, 
                                              Z = Z_nose, 
                                              y = y, 
                                              inf_feats = nose, 
                                              fit_params = fit_params, 
                                              split_params = split_params)

#Do the adaptive testing to get the p-value for the nose region
p_value_tmp_nose = cue_nose.ada_testing(X = X, 
                                        Z = Z_nose, 
                                        y = y, 
                                        ratio = ratio_nose, 
                                        perturb = perturb_nose, 
                                        fit_params = fit_params, 
                                        split_params = split_params)

In [ ]:
#Test for mouth
print('Testing for mouth')
cue_mouth = split_test(model = model, 
                       model_mask = model_mask, 
                       change = 'mask', 
                       eva_metric = 'zero-one')
#Generate Z for mouth region
Z_mouth = cue_nose.mask_cov(X, mouth)
#Check whether the region is correct
fig, axs = plt.subplots(1, 4, figsize = (25, 12))
fig.subplots_adjust(hspace = 0.2, wspace = 0.2)
axs = axs.ravel()

picture = [Z_mouth[10], Z_mouth[20], Z_mouth[30], Z_mouth[40]]
for i in range(4):
    axs[i].imshow(picture[i][:,:,0], cmap = 'gray')
    axs[i].set_xticklabels([])
    axs[i].set_yticklabels([])
    
#Generate the splitting ratio and perturb level for mouth
ratio_mouth, perturb_mouth = cue_mouth.hp_tuning(X = X, 
                                                 Z = Z_mouth, 
                                                 y = y, 
                                                 inf_feats = mouth, 
                                                 fit_params = fit_params, 
                                                 split_params = split_params)

#Do the adaptive testing to get the p-value for the mouth region
p_value_tmp_mouth = cue_mouth.ada_testing(X = X, 
                                          Z = Z_mouth, 
                                          y = y, 
                                          ratio = ratio_mouth, 
                                          perturb = perturb_mouth, 
                                          fit_params = fit_params, 
                                          split_params = split_params)